In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download valentynsichkar/traffic-signs-preprocessed

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
import os
import pickle
import zipfile
import pandas as pd
import tensorflow as tf

from keras.utils.np_utils import to_categorical

In [ ]:
zip_ref = zipfile.ZipFile('/content/data2.pickle.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
# Opening file for reading in binary mode
with open('/content/data2.pickle', 'rb') as f:
    data = pickle.load(f, encoding='latin1')  # dictionary type

# Preparing y_train and y_validation for using in Keras
data['y_train'] = to_categorical(data['y_train'], num_classes=43)
data['y_validation'] = to_categorical(data['y_validation'], num_classes=43)

# Making channels come at the end
data['x_train'] = data['x_train'].transpose(0, 2, 3, 1)
data['x_validation'] = data['x_validation'].transpose(0, 2, 3, 1)
data['x_test'] = data['x_test'].transpose(0, 2, 3, 1)

# Showing loaded data from file
for i, j in data.items():
    if i == 'labels':
        print(i + ':', len(j))
    else: 
        print(i + ':', j.shape)

y_test: (12630,)
y_validation: (4410, 43)
x_validation: (4410, 32, 32, 3)
x_train: (86989, 32, 32, 3)
y_train: (86989, 43)
labels: 43
x_test: (12630, 32, 32, 3)


In [ ]:
# Create sequential model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=15, activation='relu',input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.125),
    tf.keras.layers.Dense(43, activation='softmax')  
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Make callback function for reducing learning rate
def scheduler(epoch, lr):
    return lr * tf.math.exp(-0.1)

learningRate = tf.keras.callbacks.LearningRateScheduler(
    scheduler, verbose=0
)


In [ ]:
history = model.fit(data['x_train'], data['y_train'],
                    batch_size=5, epochs = 10,
                    validation_data = (data['x_validation'], data['y_validation']),
                    callbacks=[learningRate]
                    )

Epoch 1/20
17398/17398 [==============================] - 97s 6ms/step - loss: 0.9559 - accuracy: 0.8268 - val_loss: 0.7715 - val_accuracy: 0.8694 - lr: 9.0484e-04
Epoch 2/20
17398/17398 [==============================] - 97s 6ms/step - loss: 0.9642 - accuracy: 0.8179 - val_loss: 0.9139 - val_accuracy: 0.8170 - lr: 8.1873e-04
Epoch 3/20
17398/17398 [==============================] - 97s 6ms/step - loss: 0.8983 - accuracy: 0.8379 - val_loss: 0.9286 - val_accuracy: 0.8243 - lr: 7.4082e-04
Epoch 4/20
17398/17398 [==============================] - 98s 6ms/step - loss: 0.7818 - accuracy: 0.8528 - val_loss: 0.9580 - val_accuracy: 0.8730 - lr: 6.7032e-04
Epoch 5/20
17398/17398 [==============================] - 97s 6ms/step - loss: 0.7371 - accuracy: 0.8650 - val_loss: 0.8987 - val_accuracy: 0.7819 - lr: 6.0653e-04
Epoch 6/20
17398/17398 [==============================] - 97s 6ms/step - loss: 0.6914 - accuracy: 0.8781 - val_loss: 0.7441 - val_accuracy: 0.8882 - lr: 5.4881e-04
Epoch 7/20
17398

In [ ]:
# Defining function for getting texts for every class - labels
def label_text(file):
    # Defining list for saving label in order from 0 to 42
    label_list = []
    
    # Reading 'csv' file and getting image's labels
    r = pd.read_csv(file)
    # Going through all names
    for name in r['SignName']:
        # Adding from every row second column with name of the label
        label_list.append(name)
    
    # Returning resulted list with labels
    return label_list


# Getting labels
labels = label_text('/content/label_names.csv')


In [ ]:
# Code for predicting image
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing import image
from google.colab import files

uploaded = files.upload()
for fn in uploaded.keys():
  path = fn
  img_source = image.load_img(path, target_size = (32, 32))
  plt.imshow(img_source)
  x = image.img_to_array(img_source)
  x = np.expand_dims(x, axis = 0)

hasil = model.predict(x)

# Scores is given for image with 43 numbers of predictions for each class
# Getting only one class with maximum value
nama = np.argmax(hasil)
print('ClassId:', nama)
print('Label:', labels[nama])


ClassId: 3
Label: Batas Kecepatan (60km/h)


In [ ]:
model.save('model_final.h5')